## $P^{n}(X_{4})$ の位数

$$
P^{n}(X_{4}) = \{(\sigma_{1}, \dots, \sigma_{n}) | \sigma_{1}\sigma_{2} \cdots \sigma_{n} = 1, \sigma_{i} \neq \sigma_{j} [\forall i,j] \}
$$

## Cyclic permutations

In [1]:
def act(sigma, k)
  if k.is_a? Array
    return k.map{|v| act(sigma, v)}
  else
    ind = (sigma.index(k)+1).then{|x| (x == 4) ? 0 : x}
    return sigma[ind]
  end
end; nil

In [2]:
a = (0..3).to_a
sigma = [2,3,0,1]
tau = [3,2,0,1]
sa = act(sigma, a)
tsa = act(tau, sa)

puts "#{a}   |-#{sigma}->   #{sa}   |-#{tau}->   #{tsa}"
nil

[0, 1, 2, 3]   |-[2, 3, 0, 1]->   [1, 2, 3, 0]   |-[3, 2, 0, 1]->   [3, 0, 2, 1]


## Tuples of cyclic permutations

In [33]:
k = 4
PMLIST = (0..k-1).to_a.permutation.to_a
CPLIST = PMLIST.select{|p| p.first == 0} #(0..k-2).to_a.permutation.map{|s| s << (k-1)}

INITIAL = (0..k-1).to_a
EVENS = (0..100).map{|k| 2*k}

#p PMLIST, CPLIST
nil

<main>:1: warning: already initialized constant PMLIST
<main>:1: warning: previous definition of PMLIST was here
<main>:2: warning: already initialized constant CPLIST
<main>:2: warning: previous definition of CPLIST was here
<main>:4: warning: already initialized constant INITIAL
<main>:4: warning: previous definition of INITIAL was here
<main>:5: warning: already initialized constant EVENS
<main>:5: warning: previous definition of EVENS was here


In [4]:
CPLIST

[[0, 1, 2, 3], [0, 1, 3, 2], [0, 2, 1, 3], [0, 2, 3, 1], [0, 3, 1, 2], [0, 3, 2, 1]]

In [5]:
sigma = CPLIST[0] #[2,3,0,1]
tau = CPLIST[1] #[3,2,0,1] 

si = PMLIST.index(act(sigma, INITIAL))
tsi = PMLIST.index(act(tau, PMLIST[si]))

puts "#{INITIAL}   |-#{sigma}->   #{PMLIST[si]}   |-#{tau}->   #{PMLIST[tsi]}"
nil

[0, 1, 2, 3]   |-[0, 1, 2, 3]->   [1, 2, 3, 0]   |-[0, 1, 3, 2]->   [3, 0, 2, 1]


In [23]:
class Tree
  def initialize(pm_ind, cp_ind, depth)
    @pm_ind = pm_ind
    @cp_ind = cp_ind
    @depth = depth
    @children = (@depth > 0 && self.value == INITIAL) ? nil : []
  end
  attr_accessor :depth, :children
  
  def value
    return PMLIST[@pm_ind]
  end
  def cp
    return @cp_ind.nil? ? nil : CPLIST[@cp_ind]
  end
#-----
  
  def nodes(dp)
    if @depth == dp
      return [self]
    else 
      return (@children.nil?) ? [] : @children.map{|c| c.nodes(dp)}.flatten
    end
  end
  def terminals
    if @children.nil? 
      return []
    else
      return (@children.empty?) ? [self] : @children.map(&:terminals).flatten
    end
  end
  
  def max_depth
    return self.terminals.map(&:depth).max
  end
  
  def grow
    if (not @children.nil?) && @children.empty?
      (0..CPLIST.size-1).each do |cp_ind|
        v = act(CPLIST[cp_ind], self.value)
        pm_ind = PMLIST.index(v)
        @children << Tree.new(pm_ind, cp_ind, @depth+1)
      end
    end
  end
#-----
  
  def show
    return "< v: #{self.value}, cp: #{self.cp}, d: #{@depth} >"
  end
  def to_s
    self.show
  end
  def inspect
    self.show
  end
end; nil

In [24]:
cp_tree = Tree.new(0, nil, 0)
nil

In [90]:
max = 2*5
max.times{|d| cp_tree.nodes(d).each(&:grow) } #{|n| n.grow}}
#max.times{|d| cp_tree.nodes(d).each{|n| p n.show} }
nil

In [26]:
puts cp_tree.nodes(1)

[< v: [1, 2, 3, 0], cp: [0, 1, 2, 3], d: 1 >, < v: [1, 3, 0, 2], cp: [0, 1, 3, 2], d: 1 >, < v: [2, 3, 1, 0], cp: [0, 2, 1, 3], d: 1 >, < v: [2, 0, 3, 1], cp: [0, 2, 3, 1], d: 1 >, < v: [3, 2, 0, 1], cp: [0, 3, 1, 2], d: 1 >, < v: [3, 0, 1, 2], cp: [0, 3, 2, 1], d: 1 >]


In [91]:
max = cp_tree.max_depth
ends = EVENS[0, max/2+1].map do |d|
  nodes = cp_tree.nodes(d)
#  nsize = nodes.size
  nodes.count{|n| n.children.nil?}
end

[0, 6, 84, 2712, 90192, 3001440]

In [92]:
seq = [1]

# seq << ends[1]*seq[0]
# seq << ends[2]*seq[0] + ends[1]*seq[1]
# seq << ends[3]*seq[0] + ends[2]*seq[1] + ends[1]*seq[2]
# seq << ends[4]*seq[0] + ends[3]*seq[1] + ends[2]*seq[2] + ends[1]*seq[3]

(ends.size - 1).times do |k|
  seq << (k+1).times.map{|i| ends[i+1]*seq[k+1-(i+1)]}.sum
end
seq

[1, 6, 120, 3936, 140160, 5039616]

## Experiments

In [66]:
puts ends
puts [0] + ends.map{|v| v*6}
puts [0, 0] + ends.map{|v| v*120}
puts [0, 0, 0] + ends.map{|v| v*(2712+504+720)}
nil

[0, 6, 84, 2712, 90192]
[0, 0, 36, 504, 16272, 541152]
[0, 0, 0, 720, 10080, 325440, 10823040]
[0, 0, 0, 0, 23616, 330624, 10674432, 354995712]
